In [ ]:
import numpy as np   #final
import pandas as pd
import matplotlib.pyplot as plt
from pypots.imputation import SAITS
from sklearn.preprocessing import MinMaxScaler
!pip install xlsxwriter #final

# Training SAITS model #Final

import os
import numpy as np
import pandas as pd
from pypots.imputation.saits import SAITS

# Path to your processed Excel files
monthly_folder = "/content/drive/MyDrive/ISMR-SASTRA/monthly_excels/"
all_files = [os.path.join(monthly_folder, f) for f in os.listdir(monthly_folder) if f.endswith('.xlsx')]

day_vtec_list = []
day_labels = []

for xlsx_path in all_files:
    xls = pd.ExcelFile(xlsx_path)
    for day_sheet in xls.sheet_names:
        day_df = pd.read_excel(xls, sheet_name=day_sheet)
        # Assumption: columns are hour, vtec, and vtec is length 24 (NaN where data missing)
        vtec = day_df["vtec"].to_numpy(dtype=np.float32)
        # Pad or trim to exactly 24
        if len(vtec) < 24:
            vtec = np.pad(vtec, (0, 24-len(vtec)), constant_values=np.nan)
        elif len(vtec) > 24:
            vtec = vtec[:24]
        day_vtec_list.append(vtec)
        day_labels.append(f"{os.path.basename(xlsx_path)}_{day_sheet}")

# N_days x 24 array
data_arr = np.stack(day_vtec_list).astype("float32")
print("Shape for model:", data_arr.shape)  # (number of days, 24)

# Train SAITS for imputation
n_steps = 24
n_features = 1
n_layers = 2
d_model = 32
n_heads = 4
d_k = d_model // n_heads      # = 8
d_v = d_k                    # = 8
d_ffn = 64                   # (2 × d_model)

model = SAITS(
    n_steps=n_steps,
    n_features=n_features,
    n_layers=n_layers,
    d_model=d_model,
    n_heads=n_heads,
    d_k=d_k,
    d_v=d_v,
    d_ffn=d_ffn,
    epochs=300,
)


print("Training SAITS model on all days...")
output = model.fit({"X": data_arr[..., None]})  # Add feature axis for shape (n_days, 24, 1)
print("Done!")

# Optional: Save the trained model with pickle if you want
import pickle
with open('/content/drive/MyDrive/ISMR-SASTRA/saits_vtec_model.pkl', 'wb') as f:
    pickle.dump(model, f)
print("Trained SAITS model saved to drive.")

